# Regressão Linear com Preditores Categóricos

Com os dados de uma campanha de telemarketing, cedidos por uma companhia bancária portuguesa em 2012, pretende-se observar aqui o desempenho do ajuste de um modelo linear múltiplo a dados mistos (ora numéricos, ora categóricos) na tarefa de classificar a resposta de cada cliente, a partir dos preditores fornecidos, como 'sim' ou 'não' à oferta de aplicação num certo fundo de investimento.

Os dados utilizados, constantes na planilha bank.csv, estão disponíveis em https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

### Passo 1: Importar as bibliotecas Pandas e Numpy

In [1]:
import pandas as pd
import numpy as np

### Passo 2: Armazenar os dados da planilha num DataFrame do Pandas

Na célula a seguir, podemos visualizar parte da planilha. As colunas de índices 1,2,3,4,6,7,8,10 e 15 trazem dados categóricos que, mais adiante, serão tratados, de forma a possibilitar a modelagem dessas categorias linearmente. As demais variáveis independentes são numéricas. A última coluna é a variável dependente, 'sim' ou 'não', que queremos prever.

In [2]:
dataset = pd.read_csv('bank.csv', delimiter=';')
print(dataset)

      age            job  marital  education default  balance housing loan  \
0      30     unemployed  married    primary      no     1787      no   no   
1      33       services  married  secondary      no     4789     yes  yes   
2      35     management   single   tertiary      no     1350     yes   no   
3      30     management  married   tertiary      no     1476     yes  yes   
4      59    blue-collar  married  secondary      no        0     yes   no   
...   ...            ...      ...        ...     ...      ...     ...  ...   
4516   33       services  married  secondary      no     -333     yes   no   
4517   57  self-employed  married   tertiary     yes    -3313     yes  yes   
4518   57     technician  married  secondary      no      295      no   no   
4519   28    blue-collar  married  secondary      no     1137      no   no   
4520   44   entrepreneur   single   tertiary      no     1136     yes  yes   

       contact  day month  duration  campaign  pdays  previous 

### Passo 3: Separar variáveis independentes e variável dependente

In [3]:
X = dataset.iloc[:,0:-1].values #variáveis independentes (colunas do data frame, exceto a última)
y = dataset.iloc[:,-1].values #variável dependente (última coluna do data frame)

### Passo 4: codificar as categorias como vetores indicadores

Este é o passo mais delicado.

Para ficar claro o que faremos, tome como exemplo a penúltima coluna, 'poutcome', imaginando, por enquanto, que essa seja a única variável categórica. Ela tem quatro valores possíveis: success, failure, unknown e other. Cada uma será pensada como um dos vetores $(1,0,0,0)$, $(0,1,0,0)$, $(0,0,1,0)$ ou $(0,0,0,1)$, sendo que o $1$ numa coordenada indica o pertencimento à categoria correspondente. Por um lado, isso aumenta em três unidades a dimensão do problema: trocaríamos uma coluna por quatro colunas; por outro, isso permite modelar numericamente a relevância, para a decisão do cliente, do fato de ele ou ela encaixar-se numa ou noutra determinada categoria.

Na verdade, o objeto ColumnTransformer do código abaixo faz essa modificação com todas as variáveis categóricas: cada uma se converterá em várias colunas binárias, que depois serão justapostas, ficando mais à esquerda no data frame, por padrão.

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1,2,3,4,6,7,8,10,15])], remainder='passthrough')
# condiciona o objeto a codificar as variáveis categóricas em colunas binárias,
# sem alterar os dados das colunas numéricas

X=np.array(ct.fit_transform(X)) # rearranja X, conforme exposto acima
print(X)

[[0.0 0.0 0.0 ... 1 -1 0]
 [0.0 0.0 0.0 ... 1 339 4]
 [0.0 0.0 0.0 ... 1 330 1]
 ...
 [0.0 0.0 0.0 ... 11 -1 0]
 [0.0 1.0 0.0 ... 4 211 3]
 [0.0 0.0 1.0 ... 2 249 7]]


### Passo 5: Codificando a variável resposta

Simplesmente, cada 'yes' do vetor y vira $1$, e cada 'no' vira $0$.

In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

### Passo 6: Dividindo X e y em dois grupos: dados de treino e dados de teste

O que faz o código abaixo é separar $20\%$ da amostra para testar a capacidade de predição do nosso modelo, que será treinado nos $80\%$ restantes. É importante que não se faça nenhum tipo de normalização de escala ainda nesta etapa, para que não haja nenhum vazamento de informações sobre os dados de teste que possam enviesar a construção do modelo: os dados de teste, por assim dizer, devem ser um conjunto "novo em folha", inacessível durante o treino.

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

### Passo 7: Normalização de escala

Para que possamos comparar a magnitude dos coeficientes de regressão de uma maneira justa, é necessário medir todas as variáveis sob uma mesma "régua". No código abaixo, o objeto StandardScaler é condicionado a fazer o ajuste necessário: nos dados de treino, ele translada cada variável (coluna) pela sua média, e a divide pelo seu desvio padrão, de modo que todas as colunas fiquem com média zero e variância $1$.

Os dados de teste são lidos na mesma escala com que foram ajustados os dados de treino, para que nenhuma discrepância de ajuste altere o resultado da predição.

Obs: não é usual normalizar os zeros e uns das variáveis categóricas, visto que eles já estão, grosso modo, "normalizados": o ganho (ou perda) de qualidade de comparação nos coeficientes de regressão dessas variáveis dá-se apenas em proporção, não em ordens de grandeza. Normalizar tudo, no entanto, evita a fadiga: não queremos ter o trabalho de selecionar apenas as variáveis originalmente numéricas.

In [7]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Passo 8: Finalmente, o modelo!

Seja $n$ o número de variáveis independentes. O objetivo é encontrar os coeficientes $\beta_0, \beta_1, \ldots, \beta_n$ para os quais se possa estabelecer $$y=\beta_0+\displaystyle\sum_{j=1}^{n}\beta_jx_j$$, de maneira que a média dos erros quadráticos sobre todos os clientes fique minimizada.

O que faz o código abaixo é ajustar esses coeficientes. Em seguida, usamos esse ajuste para predizer, com os dados de teste, quais serão as entradas do vetor y_test. Armazenamos esses palpites no vetor de predições y_pred.

In [8]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test) #predições do modelo, a comparar com os dados de teste

### Mas não eram zeros e uns?

Sim. Estamos tentando prever se algo será igual a zero ou a um, sendo que esse "algo" está modelado como um número real. Só o que podemos fazer com o outcome da função de regressão é substituí-lo por zero ou por um - o que estiver mais perto. Portanto, fazemos $y = 1$ se $y > 0.5$, e $y=0$ caso contrário.

### E se apenas jogássemos uma moeda?

Seja $p$ a média de pessoas que respondem 'sim'. Qualquer modelo que se preze, para compensar todo o esforço até este ponto, deveria ter um desempenho preditivo melhor do que o de simplesmente desprezar esse amontoado de preditores e jogar uma moeda com probabilidade $p$ de sucesso para cada novo cliente.

Se fizéssemos mesmo isso, cada test case se encaixaria em uma dentre as quatro situações da tabela abaixo, com probabilidades $(1-p)^2$, $(1-p)p$, $p(1-p)$ e $p^2$.

Obs: tomamos $p$ como média dos dados de treino, novamente para que as informações do teste permaneçam "desconhecidas".

In [9]:
p = np.mean(y_train)
naive_pred = np.array([[(1-p)**2,(1-p)*p],[p*(1-p),p**2]])
naive_df = pd.DataFrame(naive_pred, columns=['Cliente rejeita', 'Cliente aceita'], index=['Moeda rejeita','Moeda aceita'])
display(naive_df)

,Cliente rejeita,Cliente aceita
Moeda rejeita,0.788049,0.099672
Moeda aceita,0.099672,0.012607


### Será que acertamos?

Agora é a hora da verdade! O que pretendemos em relação à tabela anterior é um aumento nas taxas da diagonal principal, que corresponde aos casos em que a previsão é acertada. Comparando a tabela do teste de verdade com esta outra acima, vemos um ganho significativo em pontos percentuais. Isso indica que o modelo possui realmente algum poder de predição.

In [10]:
from IPython.display import display

pred_table = np.zeros((2,2))

t = len(y_pred)
u = 1/t
for i in range(0,t):
	pred_table[int(y_pred[i]>0.5),int(y_test[i]>0.5)] += u #loop que conta os erros e acertos do modelo

df = pd.DataFrame(pred_table, columns=['Resposta Esperada \'não\'','Resposta Esperada \'sim\''], index=['Resposta Predita \'não\'','Resposta Predita \'sim\''])

display(df)

,Resposta Esperada 'não',Resposta Esperada 'sim'
Resposta Predita 'não',0.855249,0.095028
Resposta Predita 'sim',0.017680,0.032044


### Relevância dos preditores

É conveniente observarmos os coeficientes da regressão e as médias dos dados de saída (próxima célula).

Note que a variável independente tem médias bem próximas nos dados de treino e de teste. Isso é esperado, se a escolha dos dados de teste for aleatória.

Além disso, o intercepto está bem próximo dessa média. É um sinal de que a normalização de $X$ está bem feita.

Com essas três medidas tão próximas, seria possível suspeitar que a magnitude dos coeficientes das variáveis seja um tanto irrelevante, isto é, que não altere muito o resultado de cada previsão.

No entanto, não é isso o que ocorre. Como se vê a seguir, os primeiros coeficientes, que correspondem às variàveis categóricas, são da ordem de $10^{11}$ ou $10^{12}$, enquanto os demais, das variáveis numéricas, são da ordem de $10^{-2}$ ou $10^{-3}$.

Isso significa que a predição (neste modelo!) é muitíssimo mais sensível a variações categóricas. É claro que modelagens por outros métodos podem suscitar especulações de natureza muito distinta, mas a primeira pergunta que esses resultados sugerem é: **será que um modelo de classificação que desprezasse os dados numéricos, levando em conta apenas os categóricos, teria um desempenho melhor?**

Essa é uma pergunta para trabalhos subsequentes.

In [11]:
print('Média dos outcomes de treino: {:.3f}'.format(np.mean(y_train)))
print('Média dos outcomes de teste: {:.3f}'.format(np.mean(y_test)))
print('Intercepto: {:.3f}'.format(regressor.intercept_))
print('Coeficientes:')
print(regressor.coef_)

Média dos outcomes de treino: 0.112
Média dos outcomes de teste: 0.127
Intercepto: 0.113
Coeficientes:
[ 1.22454267e+12  1.64692561e+12  7.85920580e+11  6.62514644e+11
  1.67377264e+12  9.03607142e+11  7.93894521e+11  1.17553137e+12
  5.22849167e+11  1.52827586e+12  6.59253822e+11  3.68262477e+11
  4.18125918e+12  6.24932776e+12  5.64679215e+12  3.27972707e+12
  4.50986454e+12  4.12373223e+12  1.84933553e+12  2.75761837e+12
  2.75761837e+12  1.70538981e+12  1.70538981e+12 -2.06188475e+12
 -2.06188475e+12 -2.39051298e+12 -1.26148287e+12 -2.26416170e+12
 -1.68354909e+12 -2.40133660e+12 -4.51717533e+11 -1.53254211e+12
 -1.27397706e+12 -2.56829738e+12 -2.27721015e+12 -7.87630965e+11
 -3.24243032e+12 -1.98477038e+12 -9.54676033e+11 -7.53028957e+11
  4.16563238e+12  2.71722110e+12  2.24898133e+12  5.16194493e+12
  1.34860390e-03  6.27464177e-03  1.31159541e-02  1.26603317e-01
 -1.44609803e-03 -3.29589844e-03  2.89916992e-04]
